Read in the suess_works.txt trainging data and format it

In [11]:
import re

with open('seuss_works.txt', 'r', encoding='utf-8') as f:
    training_text = f.read()

pattern = r'(\w+|[^\w\s]|\s|\n)'
split = re.findall(pattern, training_text)
formatted = []
for word in split:
    if word.istitle():
        formatted.append("<C>")
        formatted.append(word.lower())
    elif word.isupper():
        formatted.append("<A>")
        formatted.append(word.lower())
    else:
        formatted.append(word)

unique = set(formatted)
print(unique)
print(len(unique))
print(f"Training Data Length: {len(formatted)}")
print(f"Token Count: {len(unique)}")

training_data = formatted
tokens = unique

['\n', ' ', '!', '"', "'", ',', '-', '.', '<A>', '<C>', '?', 'a', 'about', 'after', 'all', 'always', 'am', 'and', 'another', 'any', 'anywhere', 'are', 'as', 'asked', 'at', 'away', 'back', 'bad', 'ball', 'be', 'bed', 'bent', 'bet', 'big', 'bit', 'bite', 'boat', 'book', 'books', 'bow', 'box', 'bump', 'bumps', 'but', 'ca', 'cake', 'call', 'came', 'can', 'car', 'cat', 'cold', 'come', 'could', 'cup', 'dark', 'day', 'dear', 'deep', 'did', 'dish', 'do', 'dots', 'down', 'eat', 'eggs', 'fall', 'fan', 'fast', 'fear', 'fell', 'find', 'fish', 'fly', 'for', 'fox', 'from', 'fun', 'funny', 'game', 'games', 'gave', 'get', 'give', 'go', 'goat', 'gone', 'good', 'got', 'gown', 'green', 'had', 'hall', 'ham', 'hand', 'hands', 'has', 'hat', 'have', 'he', 'head', 'hear', 'her', 'here', 'high', 'him', 'his', 'hit', 'hold', 'home', 'hook', 'hop', 'hops', 'house', 'how', 'i', 'if', 'in', 'into', 'is', 'it', 'jump', 'jumps', 'kicks', 'kind', 'kinds', 'kite', 'kites', 'know', 'last', 'let', 'like', 'lit', 'litte'

Create an encoder and decoder

In [13]:
sorted_tokens = sorted(tokens)
encode_dict = {element: idx for idx, element in enumerate(sorted_tokens)}
decode_dict = {idx: element for idx, element in enumerate(sorted_tokens)}

def encoder(text: list[str]) -> list[int] :
    return [encode_dict[element] for element in text]

def decoder(code: list[int]) -> list[str] :
    return [decode_dict[element] for element in code]

def full_decode(code: list[str]) -> str :
    status = 0
    final = ""
    for element in code:
        if element == "<C>":
            status = 1
        elif element == "<A>":
            status = 2
        else:
            if status == 0:
                final += element
            elif status == 1:
                final += element.capitalize()
            else:
                final += element.upper()
            status = 0
    return final

print(encoder(training_data))
print(decoder(encoder(sorted_tokens)))
print(decoder(encoder(training_data)))
print(full_decode(decoder(encoder(training_data))))

[9, 90, 1, 9, 65, 1, 17, 1, 9, 93, 0, 0, 9, 115, 1, 16, 1, 9, 185, 0, 9, 185, 1, 9, 115, 1, 16, 0, 9, 217, 1, 9, 185, 6, 9, 115, 6, 16, 2, 0, 9, 217, 1, 9, 185, 6, 9, 115, 6, 16, 2, 0, 9, 115, 1, 61, 1, 155, 1, 131, 1, 217, 1, 9, 185, 6, 9, 115, 6, 16, 2, 0, 0, 9, 61, 1, 267, 1, 131, 1, 90, 1, 65, 1, 17, 1, 93, 10, 0, 9, 115, 1, 61, 1, 155, 1, 131, 1, 220, 5, 1, 9, 185, 6, 9, 115, 6, 16, 7, 0, 9, 115, 1, 61, 1, 155, 1, 131, 1, 90, 1, 65, 1, 17, 1, 93, 7, 0, 0, 9, 264, 1, 267, 1, 131, 1, 220, 1, 103, 1, 162, 1, 222, 10, 0, 0, 9, 115, 1, 264, 1, 155, 1, 131, 1, 220, 1, 103, 1, 162, 1, 222, 7, 0, 9, 115, 1, 264, 1, 155, 1, 131, 1, 220, 1, 20, 7, 0, 9, 115, 1, 61, 1, 155, 1, 131, 1, 90, 1, 65, 1, 17, 1, 93, 7, 0, 9, 115, 1, 61, 1, 155, 1, 131, 1, 220, 5, 1, 9, 185, 6, 9, 115, 6, 16, 7, 0, 0, 9, 264, 1, 267, 1, 131, 1, 220, 1, 117, 1, 11, 1, 113, 10, 0, 9, 264, 1, 267, 1, 131, 1, 220, 1, 262, 1, 11, 1, 149, 10, 0, 0, 9, 115, 1, 61, 1, 155, 1, 131, 1, 220, 1, 117, 1, 11, 1, 113, 7, 0, 9, 115

Begin Model Training

In [17]:
import torch

# Save the data as a Tensor
data = torch.tensor(encoder(training_data), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:10])

torch.Size([6224]) torch.int64
tensor([ 9, 90,  1,  9, 65,  1, 17,  1,  9, 93])


In [18]:
# Training and Validation Split
n = int(0.9*len(data))
training = data[:n]
validation = data[n:]

In [22]:
torch.manual_seed(1234)
batch_size = 4
block_size = 8
vocab_size = len(tokens)

def get_batch(type: str):
    # generate a batch of inputs x and targets y
    data = training if split == 'training' else validation
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

test_x,test_y = get_batch('training')
print('Inputs:')
print(test_x.shape)
print(test_x)
print('Targets:')
print(test_y.shape)
print(test_y)


Inputs:
torch.Size([4, 8])
tensor([[ 10,   3,   0,   0,   9,  17,   1,   9],
        [  1,  87,   5,   3,   1, 183,   1, 218],
        [119,   1, 217,   7,   3,   0,   9,  17],
        [  7,   0,   9, 115,   1, 260,   1, 198]])
Targets:
torch.Size([4, 8])
tensor([[  3,   0,   0,   9,  17,   1,   9, 184],
        [ 87,   5,   3,   1, 183,   1, 218,   1],
        [  1, 217,   7,   3,   0,   9,  17,   1],
        [  0,   9, 115,   1, 260,   1, 198,   1]])


In [26]:
# Simple Bigram Language Model

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1234)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (Batch x Text) tensors
        logits = self.token_embedding_table(idx) # (Batch x Text x Channels)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (Batch x Text) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (Batch x Channels)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (Batch x Channels)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (Batch x 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (Batch, Text + 1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(test_x, test_y)
print(logits.shape)
print(loss)

print(full_decode(decoder(m.generate(torch.zeros((1, 1), dtype=torch.long), 100)[0].tolist())))

torch.Size([32, 269])
tensor(5.9572, grad_fn=<NllLossBackward0>)

hatnotofmeatpickdidstopherewhen.andget ohshookhandsjumpsonehallnodidatopawayafterwantsaidbadfoxthinkfunpatknow!eggsto!shinefromputdownsatmindcaketrykitethetwooffunnybookthingjumpsshouldbumpspinkmebowisredthistellknownearnotgottrickslikefunareupsallyanotherfallcar!bitshinenowhamnewbookasifcake littledidpackpackbitdishgreenshowstringshasshineshowit


In [27]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [43]:
batch_size = 32
training_steps = 10000

for steps in range(training_steps):
    # get a new batch for training
    xb, yb = get_batch('training')

    # evalute the loss for the model
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

1.5684059858322144


In [46]:
# Check the predictions from the new model

print(full_decode(decoder(m.generate(torch.zeros((1, 1), dtype=torch.long), 100)[0].tolist())))


But another

He and at no shut gone
And of down..
"But this fear you cup,
Tell went the up the away.
He a picked the back that big
"
Now,"I fish..
And rake, mother What tell fish.

